# Optimization Algorithms - Comprehensive Implementation

This notebook implements various optimization algorithms with detailed visualizations and explanations.

## Table of Contents
1. Basic Gradient Descent
2. Exact Line Search
3. Armijo Rule (Backtracking)
4. Linear Regression (GD)
5. Stochastic Gradient Descent
6. Mini-Batch Gradient Descent
7. Logistic Regression
8. EWMA Adaptive Optimization
9. Subgradient Descent
10. L1 Regularization (Lasso)
11. L2 Regularization (Ridge)
12. Newton's Method (Root Finding)
13. Newton's Method (Optimization)
14. Lagrange Multiplier Method
15. KKT Conditions
16. Active Set Method

---

## Setup and Imports

First, let's import all necessary libraries for our optimization algorithms.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import minimize_scalar, minimize
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualizations
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

# Set random seed for reproducibility
np.random.seed(42)

print('All libraries imported successfully!')

---
## 1. Basic Gradient Descent

Gradient Descent is an iterative optimization algorithm for finding the minimum of a function.

**Algorithm:**
$$x_{k+1} = x_k - \alpha \nabla f(x_k)$$

Where:
- $x_k$ is the current position
- $\alpha$ is the learning rate
- $\nabla f(x_k)$ is the gradient at $x_k$

In [ ]:
def gradient_descent(f, grad_f, x0, learning_rate=0.1, max_iter=100, tol=1e-6):
    """
    Basic Gradient Descent implementation
    
    Parameters:
    -----------
    f : function
        Objective function to minimize
    grad_f : function
        Gradient of the objective function
    x0 : array-like
        Initial point
    learning_rate : float
        Step size (alpha)
    max_iter : int
        Maximum number of iterations
    tol : float
        Convergence tolerance
    
    Returns:
    --------
    x_history : list
        History of x values
    f_history : list
        History of function values
    """
    x = np.array(x0, dtype=float)
    x_history = [x.copy()]
    f_history = [f(x)]
    
    for i in range(max_iter):
        grad = grad_f(x)
        x_new = x - learning_rate * grad
        
        x_history.append(x_new.copy())
        f_history.append(f(x_new))
        
        # Check convergence
        if np.linalg.norm(x_new - x) < tol:
            break
            
        x = x_new
    
    return x_history, f_history

# Test function: Rosenbrock function
def rosenbrock(x):
    return (1 - x[0])**2 + 100 * (x[1] - x[0]**2)**2

def rosenbrock_grad(x):
    dx = -2 * (1 - x[0]) - 400 * x[0] * (x[1] - x[0]**2)
    dy = 200 * (x[1] - x[0]**2)
    return np.array([dx, dy])

# Run gradient descent
x0 = np.array([-1.0, 1.0])
x_hist, f_hist = gradient_descent(rosenbrock, rosenbrock_grad, x0, learning_rate=0.001, max_iter=1000)

print(f'Gradient Descent Results:')
print(f'Initial point: {x0}')
print(f'Final point: {x_hist[-1]}')
print(f'Final function value: {f_hist[-1]:.6f}')
print(f'Number of iterations: {len(f_hist)-1}')

In [ ]:
# Visualization for Gradient Descent
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Convergence of function value
axes[0].plot(f_hist, 'b-', linewidth=2)
axes[0].set_xlabel('Iteration', fontsize=12)
axes[0].set_ylabel('Function Value', fontsize=12)
axes[0].set_title('Gradient Descent: Convergence', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].set_yscale('log')

# Plot 2: Path on contour plot
x = np.linspace(-2, 2, 400)
y = np.linspace(-1, 3, 400)
X, Y = np.meshgrid(x, y)
Z = (1 - X)**2 + 100 * (Y - X**2)**2

axes[1].contour(X, Y, Z, levels=np.logspace(-1, 3, 20), cmap='viridis', alpha=0.6)
x_hist_arr = np.array(x_hist)
axes[1].plot(x_hist_arr[:, 0], x_hist_arr[:, 1], 'r.-', linewidth=2, markersize=8, label='GD Path')
axes[1].plot(x_hist_arr[0, 0], x_hist_arr[0, 1], 'go', markersize=12, label='Start')
axes[1].plot(x_hist_arr[-1, 0], x_hist_arr[-1, 1], 'r*', markersize=15, label='End')
axes[1].set_xlabel('x₁', fontsize=12)
axes[1].set_ylabel('x₂', fontsize=12)
axes[1].set_title('Gradient Descent: Optimization Path', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## 2. Exact Line Search

Exact line search finds the optimal step size at each iteration by minimizing the function along the search direction.

$$\alpha_k = \arg\min_{\alpha > 0} f(x_k - \alpha \nabla f(x_k))$$

In [ ]:
def exact_line_search_gd(f, grad_f, x0, max_iter=100, tol=1e-6):
    """
    Gradient Descent with Exact Line Search
    
    At each iteration, finds the optimal step size by minimizing
    the function along the gradient direction.
    """
    x = np.array(x0, dtype=float)
    x_history = [x.copy()]
    f_history = [f(x)]
    alpha_history = []
    
    for i in range(max_iter):
        grad = grad_f(x)
        
        # Define line search function
        def line_func(alpha):
            return f(x - alpha * grad)
        
        # Find optimal alpha using scipy
        result = minimize_scalar(line_func, bounds=(0, 10), method='bounded')
        alpha = result.x
        alpha_history.append(alpha)
        
        # Update x
        x_new = x - alpha * grad
        
        x_history.append(x_new.copy())
        f_history.append(f(x_new))
        
        # Check convergence
        if np.linalg.norm(x_new - x) < tol:
            break
            
        x = x_new
    
    return x_history, f_history, alpha_history

# Test with quadratic function for easier visualization
def quadratic(x):
    A = np.array([[1, 0], [0, 10]])
    return 0.5 * x.T @ A @ x

def quadratic_grad(x):
    A = np.array([[1, 0], [0, 10]])
    return A @ x

x0 = np.array([5.0, 5.0])
x_hist_els, f_hist_els, alpha_hist = exact_line_search_gd(quadratic, quadratic_grad, x0, max_iter=50)

print(f'Exact Line Search Results:')
print(f'Initial point: {x0}')
print(f'Final point: {x_hist_els[-1]}')
print(f'Final function value: {f_hist_els[-1]:.8f}')
print(f'Number of iterations: {len(f_hist_els)-1}')
print(f'Average step size: {np.mean(alpha_hist):.4f}')

In [ ]:
# Visualization for Exact Line Search
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Plot 1: Convergence comparison
axes[0].semilogy(f_hist_els, 'b-o', linewidth=2, markersize=6, label='Exact Line Search')
axes[0].set_xlabel('Iteration', fontsize=12)
axes[0].set_ylabel('Function Value (log scale)', fontsize=12)
axes[0].set_title('Exact Line Search: Convergence', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Plot 2: Step sizes
axes[1].plot(alpha_hist, 'g-o', linewidth=2, markersize=6)
axes[1].set_xlabel('Iteration', fontsize=12)
axes[1].set_ylabel('Step Size (α)', fontsize=12)
axes[1].set_title('Step Size Evolution', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

# Plot 3: Path visualization
x = np.linspace(-6, 6, 300)
y = np.linspace(-6, 6, 300)
X, Y = np.meshgrid(x, y)
Z = 0.5 * (X**2 + 10*Y**2)

axes[2].contour(X, Y, Z, levels=20, cmap='viridis', alpha=0.6)
x_hist_arr = np.array(x_hist_els)
axes[2].plot(x_hist_arr[:, 0], x_hist_arr[:, 1], 'r.-', linewidth=2, markersize=10, label='Path')
axes[2].plot(x_hist_arr[0, 0], x_hist_arr[0, 1], 'go', markersize=12, label='Start')
axes[2].plot(x_hist_arr[-1, 0], x_hist_arr[-1, 1], 'r*', markersize=15, label='End')
axes[2].set_xlabel('x₁', fontsize=12)
axes[2].set_ylabel('x₂', fontsize=12)
axes[2].set_title('Optimization Path', fontsize=14, fontweight='bold')
axes[2].legend(fontsize=10)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## 3. Armijo Rule (Backtracking Line Search)

The Armijo rule is a practical line search method that finds a step size satisfying sufficient decrease.

**Condition:**
$$f(x_k + \alpha d_k) \leq f(x_k) + c_1 \alpha \nabla f(x_k)^T d_k$$

Where $c_1 \in (0, 1)$ is typically 0.0001 to 0.3.

In [ ]:
def armijo_backtracking(f, grad_f, x0, c1=0.0001, rho=0.5, max_alpha=1.0, max_iter=100, tol=1e-6):
    """
    Gradient Descent with Armijo Backtracking Line Search
    
    Parameters:
    -----------
    c1 : float
        Armijo condition parameter (typically 0.0001 to 0.3)
    rho : float
        Backtracking factor (typically 0.5)
    max_alpha : float
        Initial step size to try
    """
    x = np.array(x0, dtype=float)
    x_history = [x.copy()]
    f_history = [f(x)]
    alpha_history = []
    backtrack_counts = []
    
    for i in range(max_iter):
        grad = grad_f(x)
        direction = -grad
        
        # Armijo backtracking
        alpha = max_alpha
        f_x = f(x)
        backtrack_count = 0
        
        while f(x + alpha * direction) > f_x + c1 * alpha * np.dot(grad, direction):
            alpha *= rho
            backtrack_count += 1
            if alpha < 1e-10:  # Prevent infinite loop
                break
        
        alpha_history.append(alpha)
        backtrack_counts.append(backtrack_count)
        
        # Update x
        x_new = x + alpha * direction
        
        x_history.append(x_new.copy())
        f_history.append(f(x_new))
        
        # Check convergence
        if np.linalg.norm(x_new - x) < tol:
            break
            
        x = x_new
    
    return x_history, f_history, alpha_history, backtrack_counts

# Test on Rosenbrock function
x0 = np.array([-1.0, 1.0])
x_hist_arm, f_hist_arm, alpha_hist_arm, bt_counts = armijo_backtracking(
    rosenbrock, rosenbrock_grad, x0, max_iter=1000
)

print(f'Armijo Backtracking Results:')
print(f'Initial point: {x0}')
print(f'Final point: {x_hist_arm[-1]}')
print(f'Final function value: {f_hist_arm[-1]:.6f}')
print(f'Number of iterations: {len(f_hist_arm)-1}')
print(f'Average backtracking steps: {np.mean(bt_counts):.2f}')
print(f'Average step size: {np.mean(alpha_hist_arm):.4f}')

In [ ]:
# Visualization for Armijo Backtracking
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Convergence
axes[0, 0].semilogy(f_hist_arm, 'b-', linewidth=2)
axes[0, 0].set_xlabel('Iteration', fontsize=12)
axes[0, 0].set_ylabel('Function Value (log scale)', fontsize=12)
axes[0, 0].set_title('Armijo Backtracking: Convergence', fontsize=14, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Step sizes
axes[0, 1].plot(alpha_hist_arm, 'g-', linewidth=2)
axes[0, 1].set_xlabel('Iteration', fontsize=12)
axes[0, 1].set_ylabel('Step Size (α)', fontsize=12)
axes[0, 1].set_title('Step Size Evolution', fontsize=14, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Backtracking steps
axes[1, 0].bar(range(len(bt_counts)), bt_counts, color='orange', alpha=0.7)
axes[1, 0].set_xlabel('Iteration', fontsize=12)
axes[1, 0].set_ylabel('Number of Backtracks', fontsize=12)
axes[1, 0].set_title('Backtracking Steps per Iteration', fontsize=14, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3, axis='y')

# Plot 4: Path on contour
x = np.linspace(-2, 2, 400)
y = np.linspace(-1, 3, 400)
X, Y = np.meshgrid(x, y)
Z = (1 - X)**2 + 100 * (Y - X**2)**2

axes[1, 1].contour(X, Y, Z, levels=np.logspace(-1, 3, 20), cmap='viridis', alpha=0.6)
x_hist_arr = np.array(x_hist_arm)
axes[1, 1].plot(x_hist_arr[:, 0], x_hist_arr[:, 1], 'r.-', linewidth=2, markersize=6, label='Path')
axes[1, 1].plot(x_hist_arr[0, 0], x_hist_arr[0, 1], 'go', markersize=12, label='Start')
axes[1, 1].plot(x_hist_arr[-1, 0], x_hist_arr[-1, 1], 'r*', markersize=15, label='End')
axes[1, 1].set_xlabel('x₁', fontsize=12)
axes[1, 1].set_ylabel('x₂', fontsize=12)
axes[1, 1].set_title('Optimization Path', fontsize=14, fontweight='bold')
axes[1, 1].legend(fontsize=10)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## 4. Linear Regression with Gradient Descent

Linear regression finds the best-fit line for data by minimizing the mean squared error.

**Objective:**
$$\min_\theta \frac{1}{2m} \sum_{i=1}^m (h_\theta(x^{(i)}) - y^{(i)})^2$$

Where $h_\theta(x) = \theta^T x$

In [ ]:
def linear_regression_gd(X, y, learning_rate=0.01, max_iter=1000, tol=1e-6):
    """
    Linear Regression using Gradient Descent
    
    Parameters:
    -----------
    X : array-like, shape (m, n)
        Training features
    y : array-like, shape (m,)
        Training labels
    """
    m, n = X.shape
    theta = np.zeros(n)
    theta_history = [theta.copy()]
    cost_history = []
    
    def compute_cost(theta):
        predictions = X @ theta
        errors = predictions - y
        return (1 / (2 * m)) * np.sum(errors ** 2)
    
    cost_history.append(compute_cost(theta))
    
    for i in range(max_iter):
        predictions = X @ theta
        errors = predictions - y
        gradient = (1 / m) * (X.T @ errors)
        
        theta_new = theta - learning_rate * gradient
        theta_history.append(theta_new.copy())
        cost_history.append(compute_cost(theta_new))
        
        if np.linalg.norm(theta_new - theta) < tol:
            break
            
        theta = theta_new
    
    return theta, theta_history, cost_history

# Generate synthetic data
np.random.seed(42)
m = 100
X_orig = 2 * np.random.rand(m, 1)
y_true = 4 + 3 * X_orig.ravel() + np.random.randn(m) * 0.5

# Add bias term
X_with_bias = np.c_[np.ones((m, 1)), X_orig]

# Run linear regression
theta_opt, theta_hist, cost_hist = linear_regression_gd(
    X_with_bias, y_true, learning_rate=0.1, max_iter=1000
)

print(f'Linear Regression Results:')
print(f'Optimal parameters: θ₀={theta_opt[0]:.4f}, θ₁={theta_opt[1]:.4f}')
print(f'True parameters: θ₀=4.0, θ₁=3.0')
print(f'Final cost: {cost_hist[-1]:.6f}')
print(f'Number of iterations: {len(cost_hist)-1}')

In [ ]:
# Visualization for Linear Regression
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Plot 1: Data and fitted line
axes[0].scatter(X_orig, y_true, alpha=0.6, s=50, color='blue', label='Data')
X_line = np.array([[0], [2]])
X_line_bias = np.c_[np.ones((2, 1)), X_line]
y_pred = X_line_bias @ theta_opt
axes[0].plot(X_line, y_pred, 'r-', linewidth=3, label=f'Fitted: y = {theta_opt[0]:.2f} + {theta_opt[1]:.2f}x')
axes[0].plot(X_line, 4 + 3*X_line, 'g--', linewidth=2, label='True: y = 4 + 3x', alpha=0.7)
axes[0].set_xlabel('x', fontsize=12)
axes[0].set_ylabel('y', fontsize=12)
axes[0].set_title('Linear Regression: Data and Fit', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Plot 2: Cost function convergence
axes[1].plot(cost_hist, 'b-', linewidth=2)
axes[1].set_xlabel('Iteration', fontsize=12)
axes[1].set_ylabel('Cost (MSE)', fontsize=12)
axes[1].set_title('Cost Function Convergence', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

# Plot 3: Parameter evolution
theta_hist_arr = np.array(theta_hist)
axes[2].plot(theta_hist_arr[:, 0], label='θ₀ (intercept)', linewidth=2)
axes[2].plot(theta_hist_arr[:, 1], label='θ₁ (slope)', linewidth=2)
axes[2].axhline(y=4, color='r', linestyle='--', alpha=0.5, label='True θ₀')
axes[2].axhline(y=3, color='g', linestyle='--', alpha=0.5, label='True θ₁')
axes[2].set_xlabel('Iteration', fontsize=12)
axes[2].set_ylabel('Parameter Value', fontsize=12)
axes[2].set_title('Parameter Evolution', fontsize=14, fontweight='bold')
axes[2].legend(fontsize=10)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## 5. Stochastic Gradient Descent (SGD)

SGD updates parameters using one sample at a time, making it faster but noisier than batch gradient descent.

**Update rule:**
$$\theta_{k+1} = \theta_k - \alpha \nabla f_i(\theta_k)$$

Where $f_i$ is the loss for a single sample.

In [ ]:
def stochastic_gradient_descent(X, y, learning_rate=0.01, max_epochs=100, shuffle=True):
    """
    Stochastic Gradient Descent for Linear Regression
    
    Updates parameters using one sample at a time
    """
    m, n = X.shape
    theta = np.zeros(n)
    theta_history = [theta.copy()]
    cost_history = []
    
    def compute_cost(theta):
        predictions = X @ theta
        errors = predictions - y
        return (1 / (2 * m)) * np.sum(errors ** 2)
    
    cost_history.append(compute_cost(theta))
    
    for epoch in range(max_epochs):
        indices = np.arange(m)
        if shuffle:
            np.random.shuffle(indices)
        
        for i in indices:
            # Gradient for single sample
            xi = X[i:i+1]
            yi = y[i:i+1]
            prediction = xi @ theta
            error = prediction - yi
            gradient = xi.T @ error
            
            theta = theta - learning_rate * gradient.ravel()
            theta_history.append(theta.copy())
        
        cost_history.append(compute_cost(theta))
    
    return theta, theta_history, cost_history

# Run SGD
theta_sgd, theta_hist_sgd, cost_hist_sgd = stochastic_gradient_descent(
    X_with_bias, y_true, learning_rate=0.01, max_epochs=50
)

print(f'Stochastic Gradient Descent Results:')
print(f'Optimal parameters: θ₀={theta_sgd[0]:.4f}, θ₁={theta_sgd[1]:.4f}')
print(f'True parameters: θ₀=4.0, θ₁=3.0')
print(f'Final cost: {cost_hist_sgd[-1]:.6f}')
print(f'Number of epochs: 50')
print(f'Total updates: {len(theta_hist_sgd)-1}')

In [ ]:
# Visualization for SGD
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Plot 1: Compare SGD vs GD convergence
axes[0].plot(cost_hist, 'b-', linewidth=2, label='Batch GD', alpha=0.7)
axes[0].plot(cost_hist_sgd, 'r-', linewidth=2, label='SGD (per epoch)', alpha=0.7)
axes[0].set_xlabel('Iteration/Epoch', fontsize=12)
axes[0].set_ylabel('Cost (MSE)', fontsize=12)
axes[0].set_title('SGD vs Batch GD: Convergence', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Plot 2: Parameter trajectory (showing noise)
theta_hist_sgd_arr = np.array(theta_hist_sgd)
# Sample every 10th point for clarity
sample_idx = np.arange(0, len(theta_hist_sgd_arr), 10)
axes[1].plot(theta_hist_sgd_arr[sample_idx, 0], theta_hist_sgd_arr[sample_idx, 1], 
             'r.-', alpha=0.5, linewidth=1, markersize=3, label='SGD path')
axes[1].plot(theta_opt[0], theta_opt[1], 'g*', markersize=15, label='Batch GD solution')
axes[1].plot(4, 3, 'b*', markersize=15, label='True parameters')
axes[1].set_xlabel('θ₀ (intercept)', fontsize=12)
axes[1].set_ylabel('θ₁ (slope)', fontsize=12)
axes[1].set_title('Parameter Space Trajectory', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

# Plot 3: Final fit comparison
axes[2].scatter(X_orig, y_true, alpha=0.6, s=50, color='blue', label='Data')
y_pred_sgd = X_line_bias @ theta_sgd
axes[2].plot(X_line, y_pred_sgd, 'r-', linewidth=3, label=f'SGD: y = {theta_sgd[0]:.2f} + {theta_sgd[1]:.2f}x')
axes[2].plot(X_line, y_pred, 'g--', linewidth=2, label=f'Batch GD', alpha=0.7)
axes[2].set_xlabel('x', fontsize=12)
axes[2].set_ylabel('y', fontsize=12)
axes[2].set_title('SGD vs Batch GD: Final Fit', fontsize=14, fontweight='bold')
axes[2].legend(fontsize=10)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## 6. Mini-Batch Gradient Descent

Mini-batch GD is a compromise between batch GD and SGD, using a small batch of samples for each update.

**Update rule:**
$$\theta_{k+1} = \theta_k - \alpha \frac{1}{|B|} \sum_{i \in B} \nabla f_i(\theta_k)$$

Where $B$ is a mini-batch of samples.

In [ ]:
def mini_batch_gradient_descent(X, y, batch_size=10, learning_rate=0.01, max_epochs=100, shuffle=True):
    """
    Mini-Batch Gradient Descent for Linear Regression
    
    Updates parameters using mini-batches of samples
    """
    m, n = X.shape
    theta = np.zeros(n)
    theta_history = [theta.copy()]
    cost_history = []
    
    def compute_cost(theta):
        predictions = X @ theta
        errors = predictions - y
        return (1 / (2 * m)) * np.sum(errors ** 2)
    
    cost_history.append(compute_cost(theta))
    
    for epoch in range(max_epochs):
        indices = np.arange(m)
        if shuffle:
            np.random.shuffle(indices)
        
        # Process mini-batches
        for start_idx in range(0, m, batch_size):
            end_idx = min(start_idx + batch_size, m)
            batch_indices = indices[start_idx:end_idx]
            
            X_batch = X[batch_indices]
            y_batch = y[batch_indices]
            
            # Gradient for mini-batch
            predictions = X_batch @ theta
            errors = predictions - y_batch
            gradient = (1 / len(batch_indices)) * (X_batch.T @ errors)
            
            theta = theta - learning_rate * gradient
            theta_history.append(theta.copy())
        
        cost_history.append(compute_cost(theta))
    
    return theta, theta_history, cost_history

# Run mini-batch GD with different batch sizes
batch_sizes = [1, 10, 32, 100]
results = {}

for bs in batch_sizes:
    theta_mb, theta_hist_mb, cost_hist_mb = mini_batch_gradient_descent(
        X_with_bias, y_true, batch_size=bs, learning_rate=0.01, max_epochs=50
    )
    results[bs] = (theta_mb, theta_hist_mb, cost_hist_mb)
    print(f'\nBatch size {bs}:')
    print(f'  Parameters: θ₀={theta_mb[0]:.4f}, θ₁={theta_mb[1]:.4f}')
    print(f'  Final cost: {cost_hist_mb[-1]:.6f}')
    print(f'  Updates per epoch: {m // bs + (1 if m % bs != 0 else 0)}')

In [ ]:
# Visualization for Mini-Batch GD
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Convergence comparison
colors = ['red', 'blue', 'green', 'purple']
for (bs, (theta_mb, _, cost_hist_mb)), color in zip(results.items(), colors):
    axes[0].plot(cost_hist_mb, linewidth=2, label=f'Batch size = {bs}', color=color, alpha=0.7)

axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Cost (MSE)', fontsize=12)
axes[0].set_title('Mini-Batch GD: Effect of Batch Size', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Plot 2: Final predictions for different batch sizes
axes[1].scatter(X_orig, y_true, alpha=0.6, s=50, color='blue', label='Data')
for (bs, (theta_mb, _, _)), color in zip(results.items(), colors):
    y_pred_mb = X_line_bias @ theta_mb
    axes[1].plot(X_line, y_pred_mb, linewidth=2, 
                label=f'BS={bs}: y = {theta_mb[0]:.2f} + {theta_mb[1]:.2f}x', 
                color=color, alpha=0.7)

axes[1].plot(X_line, 4 + 3*X_line, 'k--', linewidth=2, label='True', alpha=0.5)
axes[1].set_xlabel('x', fontsize=12)
axes[1].set_ylabel('y', fontsize=12)
axes[1].set_title('Final Fits: Different Batch Sizes', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=9)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Additional comparison plot
fig, ax = plt.subplots(figsize=(12, 6))
iterations = [len(results[bs][2]) for bs in batch_sizes]
final_costs = [results[bs][2][-1] for bs in batch_sizes]

ax.bar([str(bs) for bs in batch_sizes], final_costs, color=colors, alpha=0.7, edgecolor='black')
ax.set_xlabel('Batch Size', fontsize=12)
ax.set_ylabel('Final Cost (MSE)', fontsize=12)
ax.set_title('Final Cost vs Batch Size', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

for i, (bs, cost) in enumerate(zip(batch_sizes, final_costs)):
    ax.text(i, cost + 0.001, f'{cost:.4f}', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

---
## 7. Logistic Regression with Gradient Descent

Logistic regression is used for binary classification using the sigmoid function.

**Hypothesis:**
$$h_\theta(x) = \frac{1}{1 + e^{-\theta^T x}}$$

**Cost function (Binary Cross-Entropy):**
$$J(\theta) = -\frac{1}{m} \sum_{i=1}^m [y^{(i)} \log(h_\theta(x^{(i)})) + (1-y^{(i)}) \log(1-h_\theta(x^{(i)}))]$$

In [ ]:
def sigmoid(z):
    """Sigmoid function"""
    return 1 / (1 + np.exp(-z))

def logistic_regression_gd(X, y, learning_rate=0.1, max_iter=1000, tol=1e-6):
    """
    Logistic Regression using Gradient Descent
    """
    m, n = X.shape
    theta = np.zeros(n)
    theta_history = [theta.copy()]
    cost_history = []
    
    def compute_cost(theta):
        h = sigmoid(X @ theta)
        # Add small epsilon to prevent log(0)
        epsilon = 1e-10
        return -(1/m) * np.sum(y * np.log(h + epsilon) + (1-y) * np.log(1-h + epsilon))
    
    cost_history.append(compute_cost(theta))
    
    for i in range(max_iter):
        h = sigmoid(X @ theta)
        gradient = (1/m) * X.T @ (h - y)
        
        theta_new = theta - learning_rate * gradient
        theta_history.append(theta_new.copy())
        cost_history.append(compute_cost(theta_new))
        
        if np.linalg.norm(theta_new - theta) < tol:
            break
            
        theta = theta_new
    
    return theta, theta_history, cost_history

# Generate synthetic classification data
np.random.seed(42)
m = 200

# Class 0
X0 = np.random.randn(m//2, 2) + np.array([-2, -2])
y0 = np.zeros(m//2)

# Class 1
X1 = np.random.randn(m//2, 2) + np.array([2, 2])
y1 = np.ones(m//2)

X_cls = np.vstack([X0, X1])
y_cls = np.hstack([y0, y1])

# Add bias term
X_cls_bias = np.c_[np.ones(m), X_cls]

# Shuffle data
indices = np.random.permutation(m)
X_cls_bias = X_cls_bias[indices]
y_cls = y_cls[indices]

# Run logistic regression
theta_log, theta_hist_log, cost_hist_log = logistic_regression_gd(
    X_cls_bias, y_cls, learning_rate=0.1, max_iter=1000
)

# Calculate accuracy
predictions = sigmoid(X_cls_bias @ theta_log) >= 0.5
accuracy = np.mean(predictions == y_cls)

print(f'Logistic Regression Results:')
print(f'Final parameters: {theta_log}')
print(f'Final cost: {cost_hist_log[-1]:.6f}')
print(f'Accuracy: {accuracy*100:.2f}%')
print(f'Number of iterations: {len(cost_hist_log)-1}')

In [ ]:
# Visualization for Logistic Regression
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Cost convergence
axes[0].plot(cost_hist_log, 'b-', linewidth=2)
axes[0].set_xlabel('Iteration', fontsize=12)
axes[0].set_ylabel('Cost (Cross-Entropy)', fontsize=12)
axes[0].set_title('Logistic Regression: Cost Convergence', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Plot 2: Decision boundary
# Plot data points
axes[1].scatter(X_cls[y_cls==0, 0], X_cls[y_cls==0, 1], c='blue', alpha=0.6, s=50, label='Class 0')
axes[1].scatter(X_cls[y_cls==1, 0], X_cls[y_cls==1, 1], c='red', alpha=0.6, s=50, label='Class 1')

# Plot decision boundary
x_min, x_max = X_cls[:, 0].min() - 1, X_cls[:, 0].max() + 1
y_min, y_max = X_cls[:, 1].min() - 1, X_cls[:, 1].max() + 1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200),
                     np.linspace(y_min, y_max, 200))
Z = sigmoid(np.c_[np.ones(xx.ravel().shape), xx.ravel(), yy.ravel()] @ theta_log)
Z = Z.reshape(xx.shape)

axes[1].contour(xx, yy, Z, levels=[0.5], colors='green', linewidths=3)
axes[1].contourf(xx, yy, Z, levels=20, alpha=0.2, cmap='RdBu')
axes[1].set_xlabel('Feature 1', fontsize=12)
axes[1].set_ylabel('Feature 2', fontsize=12)
axes[1].set_title(f'Decision Boundary (Accuracy: {accuracy*100:.1f}%)', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## 8. EWMA Adaptive Optimization (Momentum)

Exponentially Weighted Moving Average (EWMA) helps accelerate gradient descent and dampen oscillations.

**Update rules:**
$$v_t = \beta v_{t-1} + (1-\beta) \nabla f(x_t)$$
$$x_{t+1} = x_t - \alpha v_t$$

Where $\beta$ is typically 0.9.

In [ ]:
def ewma_momentum_gd(f, grad_f, x0, learning_rate=0.01, beta=0.9, max_iter=1000, tol=1e-6):
    """
    Gradient Descent with EWMA Momentum
    
    Parameters:
    -----------
    beta : float
        Momentum coefficient (typically 0.9)
    """
    x = np.array(x0, dtype=float)
    v = np.zeros_like(x)  # Velocity
    
    x_history = [x.copy()]
    f_history = [f(x)]
    v_history = [v.copy()]
    
    for i in range(max_iter):
        grad = grad_f(x)
        
        # Update velocity with EWMA
        v = beta * v + (1 - beta) * grad
        
        # Update position
        x_new = x - learning_rate * v
        
        x_history.append(x_new.copy())
        f_history.append(f(x_new))
        v_history.append(v.copy())
        
        if np.linalg.norm(x_new - x) < tol:
            break
            
        x = x_new
    
    return x_history, f_history, v_history

# Test on Rosenbrock function
x0 = np.array([-1.0, 1.0])
x_hist_mom, f_hist_mom, v_hist_mom = ewma_momentum_gd(
    rosenbrock, rosenbrock_grad, x0, learning_rate=0.001, beta=0.9, max_iter=1000
)

# Compare with vanilla GD
x_hist_vanilla, f_hist_vanilla = gradient_descent(
    rosenbrock, rosenbrock_grad, x0, learning_rate=0.001, max_iter=1000
)

print(f'EWMA Momentum Results:')
print(f'Final point: {x_hist_mom[-1]}')
print(f'Final function value: {f_hist_mom[-1]:.6f}')
print(f'Iterations: {len(f_hist_mom)-1}')
print(f'\nVanilla GD:')
print(f'Final function value: {f_hist_vanilla[-1]:.6f}')
print(f'Iterations: {len(f_hist_vanilla)-1}')

In [ ]:
# Visualization for EWMA Momentum
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Convergence comparison
axes[0].semilogy(f_hist_vanilla, 'b-', linewidth=2, label='Vanilla GD', alpha=0.7)
axes[0].semilogy(f_hist_mom, 'r-', linewidth=2, label='Momentum', alpha=0.7)
axes[0].set_xlabel('Iteration', fontsize=12)
axes[0].set_ylabel('Function Value (log scale)', fontsize=12)
axes[0].set_title('EWMA Momentum vs Vanilla GD', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Plot 2: Paths comparison
x = np.linspace(-2, 2, 400)
y = np.linspace(-1, 3, 400)
X, Y = np.meshgrid(x, y)
Z = (1 - X)**2 + 100 * (Y - X**2)**2

axes[1].contour(X, Y, Z, levels=np.logspace(-1, 3, 20), cmap='viridis', alpha=0.6)
x_hist_vanilla_arr = np.array(x_hist_vanilla)
x_hist_mom_arr = np.array(x_hist_mom)

axes[1].plot(x_hist_vanilla_arr[:, 0], x_hist_vanilla_arr[:, 1], 
             'b.-', linewidth=1.5, markersize=4, label='Vanilla GD', alpha=0.7)
axes[1].plot(x_hist_mom_arr[:, 0], x_hist_mom_arr[:, 1], 
             'r.-', linewidth=1.5, markersize=4, label='Momentum', alpha=0.7)
axes[1].plot(x0[0], x0[1], 'go', markersize=12, label='Start')
axes[1].set_xlabel('x₁', fontsize=12)
axes[1].set_ylabel('x₂', fontsize=12)
axes[1].set_title('Optimization Paths Comparison', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## 9. Subgradient Descent

Subgradient descent extends gradient descent to non-differentiable functions (e.g., absolute value).

For $f(x) = |x|$, the subgradient is:
$$\partial f(x) = \begin{cases}
-1 & \text{if } x < 0 \\
[-1, 1] & \text{if } x = 0 \\
1 & \text{if } x > 0
\end{cases}$$

In [ ]:
def subgradient_descent(f, subgrad_f, x0, learning_rate=0.1, max_iter=1000):
    """
    Subgradient Descent for non-differentiable functions
    """
    x = np.array(x0, dtype=float)
    x_history = [x.copy()]
    f_history = [f(x)]
    
    for i in range(max_iter):
        subgrad = subgrad_f(x)
        
        # Diminishing step size for convergence
        alpha = learning_rate / np.sqrt(i + 1)
        
        x_new = x - alpha * subgrad
        
        x_history.append(x_new.copy())
        f_history.append(f(x_new))
        
        x = x_new
    
    return x_history, f_history

# Test function: f(x) = |x1| + |x2| (L1 norm)
def l1_norm(x):
    return np.sum(np.abs(x))

def l1_subgradient(x):
    return np.sign(x)

# Another test: f(x) = max(0, x)^2 + |x-2|
def non_smooth_func(x):
    if x.ndim == 0 or len(x) == 1:
        val = x if x.ndim == 0 else x[0]
        return max(0, val)**2 + abs(val - 2)
    else:
        return max(0, x[0])**2 + abs(x[1])

def non_smooth_subgrad(x):
    if x.ndim == 0 or len(x) == 1:
        val = x if x.ndim == 0 else x[0]
        g1 = 2*val if val > 0 else 0
        g2 = 1 if val > 2 else (-1 if val < 2 else 0)
        return np.array([g1 + g2])
    else:
        g1 = 2*x[0] if x[0] > 0 else 0
        g2 = np.sign(x[1])
        return np.array([g1, g2])

# Run subgradient descent on L1 norm
x0 = np.array([3.0, -2.0])
x_hist_sub, f_hist_sub = subgradient_descent(
    l1_norm, l1_subgradient, x0, learning_rate=1.0, max_iter=500
)

print(f'Subgradient Descent Results (L1 norm):')
print(f'Initial point: {x0}')
print(f'Final point: {x_hist_sub[-1]}')
print(f'Final function value: {f_hist_sub[-1]:.6f}')
print(f'Number of iterations: {len(f_hist_sub)-1}')

In [ ]:
# Visualization for Subgradient Descent
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Convergence
axes[0].plot(f_hist_sub, 'b-', linewidth=2)
axes[0].set_xlabel('Iteration', fontsize=12)
axes[0].set_ylabel('Function Value', fontsize=12)
axes[0].set_title('Subgradient Descent: Convergence (L1 Norm)', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].axhline(y=0, color='r', linestyle='--', label='Optimum')
axes[0].legend(fontsize=10)

# Plot 2: Path visualization
x_hist_sub_arr = np.array(x_hist_sub)
axes[1].plot(x_hist_sub_arr[:, 0], x_hist_sub_arr[:, 1], 'b.-', linewidth=2, markersize=6, label='Path')
axes[1].plot(x_hist_sub_arr[0, 0], x_hist_sub_arr[0, 1], 'go', markersize=12, label='Start')
axes[1].plot(x_hist_sub_arr[-1, 0], x_hist_sub_arr[-1, 1], 'r*', markersize=15, label='End')
axes[1].plot(0, 0, 'y*', markersize=15, label='True Optimum')

# Add contour for L1 norm
x_range = np.linspace(-4, 4, 100)
y_range = np.linspace(-3, 3, 100)
X, Y = np.meshgrid(x_range, y_range)
Z = np.abs(X) + np.abs(Y)
axes[1].contour(X, Y, Z, levels=15, cmap='viridis', alpha=0.3)

axes[1].set_xlabel('x₁', fontsize=12)
axes[1].set_ylabel('x₂', fontsize=12)
axes[1].set_title('Optimization Path on L1 Norm', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)
axes[1].axis('equal')

plt.tight_layout()
plt.show()

---
## 10. L1 Regularization (Lasso)

Lasso adds an L1 penalty to the cost function, promoting sparsity in the solution.

**Objective:**
$$\min_\theta \frac{1}{2m} \sum_{i=1}^m (h_\theta(x^{(i)}) - y^{(i)})^2 + \lambda \sum_{j=1}^n |\theta_j|$$

L1 regularization encourages some coefficients to become exactly zero.

In [ ]:
def lasso_regression(X, y, lambda_reg=0.1, learning_rate=0.01, max_iter=1000, tol=1e-6):
    """
    Lasso Regression (L1 Regularization)
    
    Uses proximal gradient descent for the L1 penalty
    """
    m, n = X.shape
    theta = np.zeros(n)
    theta_history = [theta.copy()]
    cost_history = []
    
    def compute_cost(theta):
        predictions = X @ theta
        mse = (1 / (2 * m)) * np.sum((predictions - y) ** 2)
        l1_penalty = lambda_reg * np.sum(np.abs(theta[1:]))  # Don't regularize bias
        return mse + l1_penalty
    
    cost_history.append(compute_cost(theta))
    
    def soft_threshold(x, threshold):
        """Soft-thresholding operator for L1 proximal step"""
        return np.sign(x) * np.maximum(np.abs(x) - threshold, 0)
    
    for i in range(max_iter):
        # Gradient step (without L1 term)
        predictions = X @ theta
        gradient = (1 / m) * (X.T @ (predictions - y))
        theta_temp = theta - learning_rate * gradient
        
        # Proximal step for L1 regularization (soft-thresholding)
        theta_new = theta_temp.copy()
        theta_new[1:] = soft_threshold(theta_temp[1:], learning_rate * lambda_reg)
        
        theta_history.append(theta_new.copy())
        cost_history.append(compute_cost(theta_new))
        
        if np.linalg.norm(theta_new - theta) < tol:
            break
            
        theta = theta_new
    
    return theta, theta_history, cost_history

# Generate data with sparse coefficients
np.random.seed(42)
m = 100
n = 20
X_sparse = np.random.randn(m, n-1)
true_theta = np.zeros(n-1)
true_theta[[0, 5, 10]] = [3.0, -2.0, 1.5]  # Only 3 non-zero coefficients

y_sparse = X_sparse @ true_theta + 0.5 * np.random.randn(m)
X_sparse_bias = np.c_[np.ones(m), X_sparse]

# Run Lasso with different regularization strengths
lambdas = [0.0, 0.1, 0.5, 1.0]
lasso_results = {}

for lam in lambdas:
    theta_lasso, _, cost_hist_lasso = lasso_regression(
        X_sparse_bias, y_sparse, lambda_reg=lam, learning_rate=0.01, max_iter=1000
    )
    lasso_results[lam] = theta_lasso
    
    n_nonzero = np.sum(np.abs(theta_lasso[1:]) > 1e-4)
    print(f'\nλ = {lam}:')
    print(f'  Non-zero coefficients: {n_nonzero}')
    print(f'  Top 3 coefficients: {theta_lasso[1:][[0, 5, 10]]}')
    print(f'  Final cost: {cost_hist_lasso[-1]:.4f}')

In [ ]:
# Visualization for Lasso
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Coefficient paths
for lam, theta in lasso_results.items():
    axes[0].stem(range(1, len(theta)), theta[1:], label=f'λ={lam}', 
                linefmt='-', markerfmt='o', basefmt=' ')

axes[0].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
axes[0].set_xlabel('Coefficient Index', fontsize=12)
axes[0].set_ylabel('Coefficient Value', fontsize=12)
axes[0].set_title('Lasso: Coefficient Values vs Regularization', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3, axis='y')

# Plot 2: Sparsity vs regularization
sparsity = [np.sum(np.abs(lasso_results[lam][1:]) > 1e-4) for lam in lambdas]
axes[1].plot(lambdas, sparsity, 'bo-', linewidth=2, markersize=10)
axes[1].axhline(y=3, color='r', linestyle='--', linewidth=2, label='True non-zeros')
axes[1].set_xlabel('Regularization Parameter (λ)', fontsize=12)
axes[1].set_ylabel('Number of Non-Zero Coefficients', fontsize=12)
axes[1].set_title('Sparsity vs Regularization Strength', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Additional visualization: True vs estimated coefficients
fig, ax = plt.subplots(figsize=(12, 6))
true_coef = np.concatenate([[0], true_theta])  # Add bias
x_pos = np.arange(len(true_coef))

width = 0.15
ax.bar(x_pos - 1.5*width, true_coef, width, label='True', alpha=0.8)
for i, lam in enumerate(lambdas):
    ax.bar(x_pos + (i-1.5)*width + width, lasso_results[lam], width, 
           label=f'λ={lam}', alpha=0.7)

ax.set_xlabel('Coefficient Index', fontsize=12)
ax.set_ylabel('Coefficient Value', fontsize=12)
ax.set_title('Lasso: True vs Estimated Coefficients', fontsize=14, fontweight='bold')
ax.legend(fontsize=10, ncol=5)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

---
## 11. L2 Regularization (Ridge)

Ridge regression adds an L2 penalty to prevent overfitting and improve numerical stability.

**Objective:**
$$\min_\theta \frac{1}{2m} \sum_{i=1}^m (h_\theta(x^{(i)}) - y^{(i)})^2 + \lambda \sum_{j=1}^n \theta_j^2$$

L2 regularization shrinks coefficients but doesn't make them exactly zero.

In [ ]:
def ridge_regression(X, y, lambda_reg=0.1, learning_rate=0.01, max_iter=1000, tol=1e-6):
    """
    Ridge Regression (L2 Regularization)
    """
    m, n = X.shape
    theta = np.zeros(n)
    theta_history = [theta.copy()]
    cost_history = []
    
    def compute_cost(theta):
        predictions = X @ theta
        mse = (1 / (2 * m)) * np.sum((predictions - y) ** 2)
        l2_penalty = (lambda_reg / 2) * np.sum(theta[1:] ** 2)  # Don't regularize bias
        return mse + l2_penalty
    
    cost_history.append(compute_cost(theta))
    
    for i in range(max_iter):
        predictions = X @ theta
        gradient = (1 / m) * (X.T @ (predictions - y))
        
        # Add L2 gradient (don't regularize bias term)
        l2_gradient = np.zeros_like(theta)
        l2_gradient[1:] = lambda_reg * theta[1:]
        
        theta_new = theta - learning_rate * (gradient + l2_gradient)
        
        theta_history.append(theta_new.copy())
        cost_history.append(compute_cost(theta_new))
        
        if np.linalg.norm(theta_new - theta) < tol:
            break
            
        theta = theta_new
    
    return theta, theta_history, cost_history

# Run Ridge with different regularization strengths
ridge_results = {}

for lam in lambdas:
    theta_ridge, _, cost_hist_ridge = ridge_regression(
        X_sparse_bias, y_sparse, lambda_reg=lam, learning_rate=0.01, max_iter=1000
    )
    ridge_results[lam] = theta_ridge
    
    print(f'\nλ = {lam}:')
    print(f'  Top 3 coefficients: {theta_ridge[1:][[0, 5, 10]]}')
    print(f'  L2 norm of coefficients: {np.linalg.norm(theta_ridge[1:]):.4f}')
    print(f'  Final cost: {cost_hist_ridge[-1]:.4f}')

In [ ]:
# Visualization for Ridge
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Ridge coefficient paths
for lam, theta in ridge_results.items():
    axes[0, 0].stem(range(1, len(theta)), theta[1:], label=f'λ={lam}', 
                    linefmt='-', markerfmt='o', basefmt=' ')

axes[0, 0].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
axes[0, 0].set_xlabel('Coefficient Index', fontsize=12)
axes[0, 0].set_ylabel('Coefficient Value', fontsize=12)
axes[0, 0].set_title('Ridge: Coefficient Values', fontsize=14, fontweight='bold')
axes[0, 0].legend(fontsize=10)
axes[0, 0].grid(True, alpha=0.3, axis='y')

# Plot 2: Lasso coefficient paths (for comparison)
for lam, theta in lasso_results.items():
    axes[0, 1].stem(range(1, len(theta)), theta[1:], label=f'λ={lam}', 
                    linefmt='-', markerfmt='o', basefmt=' ')

axes[0, 1].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
axes[0, 1].set_xlabel('Coefficient Index', fontsize=12)
axes[0, 1].set_ylabel('Coefficient Value', fontsize=12)
axes[0, 1].set_title('Lasso: Coefficient Values', fontsize=14, fontweight='bold')
axes[0, 1].legend(fontsize=10)
axes[0, 1].grid(True, alpha=0.3, axis='y')

# Plot 3: L2 norm vs lambda
ridge_norms = [np.linalg.norm(ridge_results[lam][1:]) for lam in lambdas]
lasso_norms = [np.linalg.norm(lasso_results[lam][1:]) for lam in lambdas]

axes[1, 0].plot(lambdas, ridge_norms, 'bo-', linewidth=2, markersize=10, label='Ridge')
axes[1, 0].plot(lambdas, lasso_norms, 'ro-', linewidth=2, markersize=10, label='Lasso')
axes[1, 0].set_xlabel('Regularization Parameter (λ)', fontsize=12)
axes[1, 0].set_ylabel('L2 Norm of Coefficients', fontsize=12)
axes[1, 0].set_title('Coefficient Shrinkage: Ridge vs Lasso', fontsize=14, fontweight='bold')
axes[1, 0].legend(fontsize=10)
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Direct comparison for λ=0.5
lam_compare = 0.5
x_pos = np.arange(1, len(ridge_results[lam_compare]))
width = 0.35

axes[1, 1].bar(x_pos - width/2, ridge_results[lam_compare][1:], width, 
               label='Ridge', alpha=0.8, color='blue')
axes[1, 1].bar(x_pos + width/2, lasso_results[lam_compare][1:], width, 
               label='Lasso', alpha=0.8, color='red')
axes[1, 1].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
axes[1, 1].set_xlabel('Coefficient Index', fontsize=12)
axes[1, 1].set_ylabel('Coefficient Value', fontsize=12)
axes[1, 1].set_title(f'Ridge vs Lasso (λ={lam_compare})', fontsize=14, fontweight='bold')
axes[1, 1].legend(fontsize=10)
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

---
## 12. Newton's Method (Root Finding)

Newton's method finds roots of equations by iteratively improving estimates.

**Update rule:**
$$x_{k+1} = x_k - \frac{f(x_k)}{f'(x_k)}$$

It has quadratic convergence near the root.

In [ ]:
def newton_root_finding(f, f_prime, x0, max_iter=50, tol=1e-10):
    """
    Newton's Method for Root Finding
    
    Finds x such that f(x) = 0
    """
    x = x0
    x_history = [x]
    f_history = [f(x)]
    
    for i in range(max_iter):
        fx = f(x)
        fpx = f_prime(x)
        
        if abs(fpx) < 1e-12:
            print("Derivative too small, stopping.")
            break
        
        x_new = x - fx / fpx
        
        x_history.append(x_new)
        f_history.append(f(x_new))
        
        if abs(x_new - x) < tol:
            break
            
        x = x_new
    
    return x_history, f_history

# Test 1: f(x) = x^2 - 2 (finding sqrt(2))
def f1(x):
    return x**2 - 2

def f1_prime(x):
    return 2*x

x_hist_n1, f_hist_n1 = newton_root_finding(f1, f1_prime, x0=1.0, max_iter=10)

print(f'Newton Root Finding (sqrt(2)):')
print(f'Initial guess: 1.0')
print(f'Final root: {x_hist_n1[-1]:.10f}')
print(f'True value: {np.sqrt(2):.10f}')
print(f'Iterations: {len(x_hist_n1)-1}')

# Test 2: f(x) = x^3 - x - 2
def f2(x):
    return x**3 - x - 2

def f2_prime(x):
    return 3*x**2 - 1

x_hist_n2, f_hist_n2 = newton_root_finding(f2, f2_prime, x0=2.0, max_iter=10)

print(f'\nNewton Root Finding (x^3 - x - 2 = 0):')
print(f'Initial guess: 2.0')
print(f'Final root: {x_hist_n2[-1]:.10f}')
print(f'f(root) = {f2(x_hist_n2[-1]):.2e}')
print(f'Iterations: {len(x_hist_n2)-1}')

In [ ]:
# Visualization for Newton's Root Finding
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Convergence for sqrt(2)
errors1 = np.abs(np.array(x_hist_n1) - np.sqrt(2))
axes[0, 0].semilogy(errors1, 'bo-', linewidth=2, markersize=8)
axes[0, 0].set_xlabel('Iteration', fontsize=12)
axes[0, 0].set_ylabel('Absolute Error (log scale)', fontsize=12)
axes[0, 0].set_title('Newton Root Finding: Convergence (sqrt(2))', fontsize=14, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Visualization of Newton's method
x_range = np.linspace(0.5, 2.5, 200)
y_range = f1(x_range)

axes[0, 1].plot(x_range, y_range, 'b-', linewidth=2, label='f(x) = x² - 2')
axes[0, 1].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
axes[0, 1].axvline(x=np.sqrt(2), color='red', linestyle='--', linewidth=2, label='True root', alpha=0.5)

# Show tangent lines for first few iterations
for i in range(min(4, len(x_hist_n1)-1)):
    x_i = x_hist_n1[i]
    y_i = f1(x_i)
    slope = f1_prime(x_i)
    
    # Tangent line
    x_tan = np.array([x_i - 0.5, x_i + 0.5])
    y_tan = y_i + slope * (x_tan - x_i)
    axes[0, 1].plot(x_tan, y_tan, 'g--', alpha=0.5, linewidth=1)
    axes[0, 1].plot(x_i, y_i, 'ro', markersize=8)
    axes[0, 1].plot([x_hist_n1[i+1], x_hist_n1[i+1]], [0, f1(x_hist_n1[i+1])], 
                   'r--', alpha=0.3, linewidth=1)

axes[0, 1].set_xlabel('x', fontsize=12)
axes[0, 1].set_ylabel('f(x)', fontsize=12)
axes[0, 1].set_title('Newton Method Iterations (First 4 steps)', fontsize=14, fontweight='bold')
axes[0, 1].legend(fontsize=10)
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].set_ylim([-1, 3])

# Plot 3: Convergence for cubic
axes[1, 0].semilogy(np.abs(f_hist_n2), 'ro-', linewidth=2, markersize=8)
axes[1, 0].set_xlabel('Iteration', fontsize=12)
axes[1, 0].set_ylabel('|f(x)| (log scale)', fontsize=12)
axes[1, 0].set_title('Newton Root Finding: |f(x)| vs Iteration', fontsize=14, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Cubic function
x_range2 = np.linspace(0, 3, 200)
y_range2 = f2(x_range2)

axes[1, 1].plot(x_range2, y_range2, 'b-', linewidth=2, label='f(x) = x³ - x - 2')
axes[1, 1].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
axes[1, 1].plot(x_hist_n2, [f2(x) for x in x_hist_n2], 'ro-', markersize=8, label='Newton iterates')
axes[1, 1].plot(x_hist_n2[-1], 0, 'g*', markersize=15, label='Final root')
axes[1, 1].set_xlabel('x', fontsize=12)
axes[1, 1].set_ylabel('f(x)', fontsize=12)
axes[1, 1].set_title('Cubic Function Root Finding', fontsize=14, fontweight='bold')
axes[1, 1].legend(fontsize=10)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## 13. Newton's Method (Optimization)

Newton's method for optimization finds stationary points by solving $\nabla f(x) = 0$.

**Update rule:**
$$x_{k+1} = x_k - [\nabla^2 f(x_k)]^{-1} \nabla f(x_k)$$

Where $\nabla^2 f$ is the Hessian matrix. Has quadratic convergence.

In [ ]:
def newton_optimization(f, grad_f, hessian_f, x0, max_iter=50, tol=1e-8):
    """
    Newton's Method for Optimization
    
    Finds x that minimizes f(x)
    """
    x = np.array(x0, dtype=float)
    x_history = [x.copy()]
    f_history = [f(x)]
    
    for i in range(max_iter):
        grad = grad_f(x)
        hess = hessian_f(x)
        
        try:
            # Solve: Hessian * delta = -gradient
            delta = np.linalg.solve(hess, -grad)
        except np.linalg.LinAlgError:
            print("Singular Hessian, stopping.")
            break
        
        x_new = x + delta
        
        x_history.append(x_new.copy())
        f_history.append(f(x_new))
        
        if np.linalg.norm(delta) < tol:
            break
            
        x = x_new
    
    return x_history, f_history

# Quadratic function (exact in 1 iteration)
def hessian_quad(x):
    return np.array([[1, 0], [0, 10]])

x0 = np.array([5.0, 5.0])
x_hist_newt, f_hist_newt = newton_optimization(
    quadratic, quadratic_grad, hessian_quad, x0, max_iter=10
)

print(f'Newton Optimization (Quadratic):')
print(f'Initial point: {x0}')
print(f'Final point: {x_hist_newt[-1]}')
print(f'Final function value: {f_hist_newt[-1]:.10f}')
print(f'Iterations: {len(f_hist_newt)-1}')

# Rosenbrock function
def rosenbrock_hessian(x):
    h11 = 2 - 400*x[1] + 1200*x[0]**2
    h12 = -400*x[0]
    h21 = -400*x[0]
    h22 = 200
    return np.array([[h11, h12], [h21, h22]])

x0_ros = np.array([0.5, 0.5])
x_hist_newt_ros, f_hist_newt_ros = newton_optimization(
    rosenbrock, rosenbrock_grad, rosenbrock_hessian, x0_ros, max_iter=50
)

print(f'\nNewton Optimization (Rosenbrock):')
print(f'Initial point: {x0_ros}')
print(f'Final point: {x_hist_newt_ros[-1]}')
print(f'Final function value: {f_hist_newt_ros[-1]:.10f}')
print(f'Iterations: {len(f_hist_newt_ros)-1}')

In [ ]:
# Visualization for Newton's Optimization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Convergence comparison (Quadratic)
axes[0, 0].semilogy(f_hist_els, 'b-', linewidth=2, label='Exact Line Search', alpha=0.7)
axes[0, 0].semilogy(f_hist_newt, 'r-o', linewidth=2, markersize=8, label='Newton', alpha=0.7)
axes[0, 0].set_xlabel('Iteration', fontsize=12)
axes[0, 0].set_ylabel('Function Value (log scale)', fontsize=12)
axes[0, 0].set_title('Newton vs Line Search (Quadratic)', fontsize=14, fontweight='bold')
axes[0, 0].legend(fontsize=10)
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Path comparison (Quadratic)
x = np.linspace(-6, 6, 300)
y = np.linspace(-6, 6, 300)
X, Y = np.meshgrid(x, y)
Z = 0.5 * (X**2 + 10*Y**2)

axes[0, 1].contour(X, Y, Z, levels=20, cmap='viridis', alpha=0.6)
x_hist_newt_arr = np.array(x_hist_newt)
axes[0, 1].plot(x_hist_newt_arr[:, 0], x_hist_newt_arr[:, 1], 
                'r.-', linewidth=2, markersize=10, label='Newton Path')
axes[0, 1].plot(x_hist_newt_arr[0, 0], x_hist_newt_arr[0, 1], 'go', markersize=12, label='Start')
axes[0, 1].plot(x_hist_newt_arr[-1, 0], x_hist_newt_arr[-1, 1], 'r*', markersize=15, label='End')
axes[0, 1].set_xlabel('x₁', fontsize=12)
axes[0, 1].set_ylabel('x₂', fontsize=12)
axes[0, 1].set_title('Newton Path (Quadratic)', fontsize=14, fontweight='bold')
axes[0, 1].legend(fontsize=10)
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Convergence (Rosenbrock)
axes[1, 0].semilogy(f_hist_newt_ros, 'r-o', linewidth=2, markersize=6)
axes[1, 0].set_xlabel('Iteration', fontsize=12)
axes[1, 0].set_ylabel('Function Value (log scale)', fontsize=12)
axes[1, 0].set_title('Newton Optimization (Rosenbrock)', fontsize=14, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Path (Rosenbrock)
x = np.linspace(-0.5, 1.5, 400)
y = np.linspace(-0.5, 1.5, 400)
X, Y = np.meshgrid(x, y)
Z = (1 - X)**2 + 100 * (Y - X**2)**2

axes[1, 1].contour(X, Y, Z, levels=np.logspace(-1, 2, 20), cmap='viridis', alpha=0.6)
x_hist_newt_ros_arr = np.array(x_hist_newt_ros)
axes[1, 1].plot(x_hist_newt_ros_arr[:, 0], x_hist_newt_ros_arr[:, 1], 
                'r.-', linewidth=2, markersize=8, label='Newton Path')
axes[1, 1].plot(x_hist_newt_ros_arr[0, 0], x_hist_newt_ros_arr[0, 1], 
                'go', markersize=12, label='Start')
axes[1, 1].plot(1, 1, 'y*', markersize=15, label='True Optimum')
axes[1, 1].set_xlabel('x₁', fontsize=12)
axes[1, 1].set_ylabel('x₂', fontsize=12)
axes[1, 1].set_title('Newton Path (Rosenbrock)', fontsize=14, fontweight='bold')
axes[1, 1].legend(fontsize=10)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
## 14. Lagrange Multiplier Method

Lagrange multipliers solve constrained optimization problems.

**Problem:**
$$\min_x f(x) \quad \text{subject to} \quad g(x) = 0$$

**Lagrangian:**
$$\mathcal{L}(x, \lambda) = f(x) + \lambda g(x)$$

**Optimality conditions:**
$$\nabla_x \mathcal{L} = 0, \quad g(x) = 0$$

In [ ]:
def lagrange_multiplier_example():
    """
    Example: Minimize f(x, y) = x^2 + y^2
    Subject to: g(x, y) = x + y - 1 = 0
    
    Solution: x* = y* = 0.5, lambda* = -1
    """
    # Analytical solution
    x_opt = 0.5
    y_opt = 0.5
    lambda_opt = -1.0
    
    print(f'Lagrange Multiplier Example:')
    print(f'Minimize: f(x,y) = x² + y²')
    print(f'Subject to: x + y = 1')
    print(f'\nAnalytical Solution:')
    print(f'x* = {x_opt}, y* = {y_opt}')
    print(f'λ* = {lambda_opt}')
    print(f'f(x*, y*) = {x_opt**2 + y_opt**2}')
    
    return x_opt, y_opt, lambda_opt

def lagrange_numerical(f, grad_f, g, grad_g, x0, lambda0, learning_rate=0.1, max_iter=1000, tol=1e-6):
    """
    Solve constrained optimization using gradient descent on Lagrangian
    """
    x = np.array(x0, dtype=float)
    lam = lambda0
    
    x_history = [x.copy()]
    lambda_history = [lam]
    f_history = [f(x)]
    constraint_history = [g(x)]
    
    for i in range(max_iter):
        # Gradient of Lagrangian w.r.t. x
        grad_L_x = grad_f(x) + lam * grad_g(x)
        
        # Update x (gradient descent)
        x = x - learning_rate * grad_L_x
        
        # Update lambda (gradient ascent on dual)
        lam = lam + learning_rate * g(x)
        
        x_history.append(x.copy())
        lambda_history.append(lam)
        f_history.append(f(x))
        constraint_history.append(g(x))
        
        if np.linalg.norm(grad_L_x) < tol and abs(g(x)) < tol:
            break
    
    return x_history, lambda_history, f_history, constraint_history

# Example problem
def f_lag(x):
    return x[0]**2 + x[1]**2

def grad_f_lag(x):
    return 2 * x

def g_lag(x):
    return x[0] + x[1] - 1

def grad_g_lag(x):
    return np.array([1.0, 1.0])

# Analytical solution
x_opt, y_opt, lambda_opt = lagrange_multiplier_example()

# Numerical solution
x0 = np.array([2.0, 2.0])
lambda0 = 0.0

x_hist_lag, lam_hist_lag, f_hist_lag, c_hist_lag = lagrange_numerical(
    f_lag, grad_f_lag, g_lag, grad_g_lag, x0, lambda0, learning_rate=0.1, max_iter=1000
)

print(f'\nNumerical Solution:')
print(f'x* = {x_hist_lag[-1]}')
print(f'λ* = {lam_hist_lag[-1]:.4f}')
print(f'f(x*) = {f_hist_lag[-1]:.6f}')
print(f'Constraint violation: {abs(c_hist_lag[-1]):.2e}')
print(f'Iterations: {len(f_hist_lag)-1}')

In [ ]:
# Visualization for Lagrange Multipliers
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Objective function convergence
axes[0, 0].plot(f_hist_lag, 'b-', linewidth=2)
axes[0, 0].axhline(y=0.5, color='r', linestyle='--', linewidth=2, label='True optimum')
axes[0, 0].set_xlabel('Iteration', fontsize=12)
axes[0, 0].set_ylabel('Objective f(x)', fontsize=12)
axes[0, 0].set_title('Objective Function Convergence', fontsize=14, fontweight='bold')
axes[0, 0].legend(fontsize=10)
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Constraint violation
axes[0, 1].semilogy(np.abs(c_hist_lag), 'r-', linewidth=2)
axes[0, 1].set_xlabel('Iteration', fontsize=12)
axes[0, 1].set_ylabel('|g(x)| (log scale)', fontsize=12)
axes[0, 1].set_title('Constraint Violation', fontsize=14, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Lambda evolution
axes[1, 0].plot(lam_hist_lag, 'g-', linewidth=2)
axes[1, 0].axhline(y=-1, color='r', linestyle='--', linewidth=2, label='True λ*')
axes[1, 0].set_xlabel('Iteration', fontsize=12)
axes[1, 0].set_ylabel('Lagrange Multiplier λ', fontsize=12)
axes[1, 0].set_title('Multiplier Evolution', fontsize=14, fontweight='bold')
axes[1, 0].legend(fontsize=10)
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Contour plot with constraint
x_range = np.linspace(-0.5, 2.5, 300)
y_range = np.linspace(-0.5, 2.5, 300)
X, Y = np.meshgrid(x_range, y_range)
Z = X**2 + Y**2

axes[1, 1].contour(X, Y, Z, levels=20, cmap='viridis', alpha=0.6)

# Constraint line: x + y = 1
x_constraint = np.linspace(-0.5, 2.5, 100)
y_constraint = 1 - x_constraint
axes[1, 1].plot(x_constraint, y_constraint, 'k-', linewidth=3, label='Constraint: x+y=1')

# Optimization path
x_hist_lag_arr = np.array(x_hist_lag)
axes[1, 1].plot(x_hist_lag_arr[:, 0], x_hist_lag_arr[:, 1], 
                'r.-', linewidth=2, markersize=6, label='Path', alpha=0.7)
axes[1, 1].plot(x_hist_lag_arr[0, 0], x_hist_lag_arr[0, 1], 'go', markersize=12, label='Start')
axes[1, 1].plot(0.5, 0.5, 'r*', markersize=15, label='Optimum')

axes[1, 1].set_xlabel('x', fontsize=12)
axes[1, 1].set_ylabel('y', fontsize=12)
axes[1, 1].set_title('Constrained Optimization', fontsize=14, fontweight='bold')
axes[1, 1].legend(fontsize=10)
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].set_xlim([-0.5, 2.5])
axes[1, 1].set_ylim([-0.5, 2.5])

plt.tight_layout()
plt.show()

---
## 15. KKT Conditions

Karush-Kuhn-Tucker (KKT) conditions generalize Lagrange multipliers to inequality constraints.

**Problem:**
$$\min_x f(x) \quad \text{s.t.} \quad g_i(x) \leq 0, \; h_j(x) = 0$$

**KKT Conditions:**
1. Stationarity: $\nabla f(x^*) + \sum \mu_i \nabla g_i(x^*) + \sum \lambda_j \nabla h_j(x^*) = 0$
2. Primal feasibility: $g_i(x^*) \leq 0, \; h_j(x^*) = 0$
3. Dual feasibility: $\mu_i \geq 0$
4. Complementary slackness: $\mu_i g_i(x^*) = 0$

In [ ]:
def kkt_example():
    """
    Example: Minimize f(x, y) = x^2 + y^2
    Subject to: x + y >= 1 (or -x - y + 1 <= 0)
                x >= 0, y >= 0
    
    This is the same as the Lagrange example but with inequality.
    """
    print('KKT Conditions Example:')
    print('Minimize: f(x,y) = x² + y²')
    print('Subject to: x + y >= 1, x >= 0, y >= 0')
    print('\nAnalytical Solution:')
    print('x* = 0.5, y* = 0.5')
    print('μ* = 1.0 (multiplier for x+y>=1)')
    print('Constraint is active: x + y = 1')
    print('\nKKT Conditions Verification:')
    print('1. Stationarity: ∇f + μ∇g = 0')
    print('   [2x, 2y] + μ[-1, -1] = [0, 0]')
    print('   [1, 1] + 1*[-1, -1] = [0, 0] ✓')
    print('2. Primal feasibility: x+y >= 1 ✓')
    print('3. Dual feasibility: μ >= 0 ✓')
    print('4. Complementary slackness: μ(1-x-y) = 0 ✓')

kkt_example()

def visualize_kkt_regions():
    """
    Visualize feasible region and KKT conditions
    """
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Create grid
    x_range = np.linspace(-0.5, 2, 300)
    y_range = np.linspace(-0.5, 2, 300)
    X, Y = np.meshgrid(x_range, y_range)
    Z = X**2 + Y**2
    
    # Plot 1: Feasible region
    axes[0].contour(X, Y, Z, levels=15, cmap='viridis', alpha=0.6)
    
    # Feasible region: x >= 0, y >= 0, x + y >= 1
    x_constraint = np.linspace(0, 2, 100)
    y_constraint = 1 - x_constraint
    axes[0].fill_between(x_constraint, y_constraint, 2, 
                        where=(y_constraint >= 0), alpha=0.3, color='green', label='Feasible region')
    axes[0].plot(x_constraint, y_constraint, 'k-', linewidth=3, label='Active: x+y=1')
    axes[0].plot(0.5, 0.5, 'r*', markersize=20, label='Optimum (KKT point)')
    
    # Gradient at optimum
    grad_f = np.array([2*0.5, 2*0.5])
    axes[0].arrow(0.5, 0.5, -grad_f[0]*0.3, -grad_f[1]*0.3, 
                 head_width=0.1, head_length=0.1, fc='red', ec='red', linewidth=2, label='−∇f')
    
    # Normal to constraint
    normal = np.array([-1, -1])  # gradient of -x-y+1
    axes[0].arrow(0.5, 0.5, normal[0]*0.3, normal[1]*0.3, 
                 head_width=0.1, head_length=0.1, fc='blue', ec='blue', linewidth=2, label='∇g')
    
    axes[0].set_xlabel('x', fontsize=12)
    axes[0].set_ylabel('y', fontsize=12)
    axes[0].set_title('KKT: Feasible Region and Optimum', fontsize=14, fontweight='bold')
    axes[0].legend(fontsize=10)
    axes[0].grid(True, alpha=0.3)
    axes[0].set_xlim([-0.5, 2])
    axes[0].set_ylim([-0.5, 2])
    
    # Plot 2: Complementary slackness visualization
    x_vals = np.linspace(0, 2, 100)
    y_vals = np.linspace(0, 2, 100)
    X2, Y2 = np.meshgrid(x_vals, y_vals)
    
    # Constraint: g(x,y) = 1 - x - y
    G = 1 - X2 - Y2
    
    # Multiplier (would be positive when constraint is active)
    mu = np.where(G >= 0, 1.0, 0.0)  # Simplified
    
    # Complementary slackness: μ * g = 0
    cs_product = mu * G
    
    im = axes[1].contourf(X2, Y2, cs_product, levels=20, cmap='RdYlGn')
    axes[1].plot(x_constraint, y_constraint, 'k-', linewidth=3, label='Constraint boundary')
    axes[1].plot(0.5, 0.5, 'r*', markersize=20, label='Optimum')
    
    axes[1].set_xlabel('x', fontsize=12)
    axes[1].set_ylabel('y', fontsize=12)
    axes[1].set_title('Complementary Slackness: μ·g(x)', fontsize=14, fontweight='bold')
    axes[1].legend(fontsize=10)
    axes[1].grid(True, alpha=0.3)
    plt.colorbar(im, ax=axes[1])
    
    plt.tight_layout()
    plt.show()

visualize_kkt_regions()

---
## 16. Active Set Method

Active Set methods solve inequality-constrained optimization by iteratively identifying which constraints are active.

**Algorithm:**
1. Start with a guess of active constraints
2. Solve equality-constrained problem with active constraints
3. Check KKT conditions
4. Update active set if needed
5. Repeat until convergence

In [ ]:
def active_set_qp_simple():
    """
    Simple demonstration of Active Set method concept
    
    Minimize: f(x, y) = (x-2)^2 + (y-1)^2
    Subject to: x >= 0, y >= 0, x + y <= 2
    """
    print('Active Set Method Example:')
    print('Minimize: f(x,y) = (x-2)² + (y-1)²')
    print('Subject to: x >= 0, y >= 0, x + y <= 2')
    print('\nUnconstrained optimum: (2, 1)')
    print('This violates x + y <= 2, so constraint is active')
    print('\nSolving with active constraint x + y = 2:')
    print('Minimize (x-2)² + (y-1)² subject to x + y = 2')
    
    # Using Lagrange multipliers for x + y = 2
    # L = (x-2)^2 + (y-1)^2 + λ(x + y - 2)
    # dL/dx = 2(x-2) + λ = 0  =>  x = 2 - λ/2
    # dL/dy = 2(y-1) + λ = 0  =>  y = 1 - λ/2
    # x + y = 2  =>  3 - λ = 2  =>  λ = 1
    
    x_opt = 2 - 1/2
    y_opt = 1 - 1/2
    lambda_opt = 1
    
    print(f'\nConstrained optimum: x* = {x_opt}, y* = {y_opt}')
    print(f'Lagrange multiplier: λ* = {lambda_opt}')
    print(f'Objective value: {(x_opt-2)**2 + (y_opt-1)**2:.4f}')
    print(f'\nActive constraints: {{x + y <= 2}}')
    print(f'Inactive constraints: {{x >= 0, y >= 0}}')
    
    return x_opt, y_opt, lambda_opt

x_opt_as, y_opt_as, lambda_opt_as = active_set_qp_simple()

In [ ]:
# Visualization for Active Set Method
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Create grid
x_range = np.linspace(-0.5, 3, 300)
y_range = np.linspace(-0.5, 3, 300)
X, Y = np.meshgrid(x_range, y_range)
Z = (X - 2)**2 + (Y - 1)**2

# Plot 1: Active Set visualization
axes[0].contour(X, Y, Z, levels=20, cmap='viridis', alpha=0.6)

# Feasible region
x_constraint = np.linspace(0, 2, 100)
y_constraint_upper = 2 - x_constraint
axes[0].fill_between(x_constraint, 0, y_constraint_upper, 
                     alpha=0.3, color='green', label='Feasible region')

# Constraint boundaries
axes[0].plot([0, 0], [0, 3], 'k--', linewidth=2, alpha=0.5, label='x = 0')
axes[0].plot([0, 3], [0, 0], 'k--', linewidth=2, alpha=0.5, label='y = 0')
axes[0].plot(x_constraint, y_constraint_upper, 'k-', linewidth=3, label='Active: x+y=2')

# Optima
axes[0].plot(2, 1, 'b*', markersize=20, label='Unconstrained opt')
axes[0].plot(x_opt_as, y_opt_as, 'r*', markersize=20, label='Constrained opt')

# Path from unconstrained to constrained
axes[0].arrow(2, 1, x_opt_as-2, y_opt_as-1, 
             head_width=0.1, head_length=0.1, fc='purple', ec='purple', linewidth=2)

axes[0].set_xlabel('x', fontsize=12)
axes[0].set_ylabel('y', fontsize=12)
axes[0].set_title('Active Set: Feasible Region', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)
axes[0].set_xlim([-0.5, 3])
axes[0].set_ylim([-0.5, 3])

# Plot 2: Active Set algorithm steps (conceptual)
steps = ['Initial\nGuess', 'Check\nConstraints', 'Update\nActive Set', 
         'Solve EQP', 'Check\nKKT', 'Converged']
y_pos = np.arange(len(steps))

axes[1].barh(y_pos, [1]*len(steps), color=['lightblue', 'lightgreen', 'lightyellow', 
                                            'lightcoral', 'lightgreen', 'lightblue'], 
             edgecolor='black', linewidth=2)

for i, step in enumerate(steps):
    axes[1].text(0.5, i, step, ha='center', va='center', fontsize=11, fontweight='bold')

# Add arrows
for i in range(len(steps)-1):
    axes[1].annotate('', xy=(0.5, i+1), xytext=(0.5, i),
                    arrowprops=dict(arrowstyle='->', lw=2, color='black'))

axes[1].set_yticks([])
axes[1].set_xticks([])
axes[1].set_xlim([0, 1])
axes[1].set_title('Active Set Algorithm Steps', fontsize=14, fontweight='bold')
axes[1].spines['top'].set_visible(False)
axes[1].spines['right'].set_visible(False)
axes[1].spines['bottom'].set_visible(False)
axes[1].spines['left'].set_visible(False)

plt.tight_layout()
plt.show()

---
## Summary and Comparison

Let's create a comprehensive comparison of all the optimization methods we've implemented.

In [ ]:
# Summary comparison
summary_data = {
    'Algorithm': [
        'Gradient Descent',
        'Exact Line Search',
        'Armijo Backtracking',
        'Linear Regression (GD)',
        'SGD',
        'Mini-Batch GD',
        'Logistic Regression',
        'EWMA Momentum',
        'Subgradient Descent',
        'Lasso (L1)',
        'Ridge (L2)',
        "Newton's Method (Root)",
        "Newton's Method (Opt)",
        'Lagrange Multipliers',
        'KKT Conditions',
        'Active Set Method'
    ],
    'Type': [
        'Unconstrained',
        'Unconstrained',
        'Unconstrained',
        'Unconstrained',
        'Unconstrained',
        'Unconstrained',
        'Unconstrained',
        'Unconstrained',
        'Non-smooth',
        'Regularized',
        'Regularized',
        'Root Finding',
        'Second-order',
        'Equality Const.',
        'Inequality Const.',
        'Inequality Const.'
    ],
    'Convergence': [
        'Linear',
        'Linear',
        'Linear',
        'Linear',
        'Sublinear',
        'Linear',
        'Linear',
        'Faster than GD',
        'Sublinear',
        'Linear',
        'Linear',
        'Quadratic',
        'Quadratic',
        'Problem-dependent',
        'Problem-dependent',
        'Finite steps (QP)'
    ],
    'Key Feature': [
        'Fixed step size',
        'Optimal step size',
        'Adaptive step size',
        'Least squares',
        'Sample-by-sample',
        'Batch updates',
        'Binary classification',
        'Accelerated',
        'Non-differentiable',
        'Sparse solutions',
        'Smooth shrinkage',
        'Very fast near root',
        'Uses Hessian',
        'Equality constraints',
        'Inequality + equality',
        'Identifies active set'
    ]
}

import pandas as pd
df_summary = pd.DataFrame(summary_data)
print('\n' + '='*80)
print('OPTIMIZATION ALGORITHMS SUMMARY')
print('='*80 + '\n')
print(df_summary.to_string(index=False))
print('\n' + '='*80)

In [ ]:
# Final visualization: Algorithm categories
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Algorithm types pie chart
type_counts = df_summary['Type'].value_counts()
axes[0, 0].pie(type_counts.values, labels=type_counts.index, autopct='%1.1f%%',
               startangle=90, colors=sns.color_palette('Set3'))
axes[0, 0].set_title('Algorithms by Type', fontsize=14, fontweight='bold')

# Plot 2: Convergence rates
conv_counts = df_summary['Convergence'].value_counts()
axes[0, 1].bar(range(len(conv_counts)), conv_counts.values, 
               color=sns.color_palette('Set2'), edgecolor='black', linewidth=1.5)
axes[0, 1].set_xticks(range(len(conv_counts)))
axes[0, 1].set_xticklabels(conv_counts.index, rotation=45, ha='right')
axes[0, 1].set_ylabel('Count', fontsize=12)
axes[0, 1].set_title('Convergence Rates', fontsize=14, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3, axis='y')

# Plot 3: Timeline/categorization
categories = ['First-Order\nMethods', 'Second-Order\nMethods', 
              'Regularized\nMethods', 'Constrained\nOptimization']
cat_counts = [8, 2, 2, 4]  # Manual count

colors_cat = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A']
axes[1, 0].barh(categories, cat_counts, color=colors_cat, edgecolor='black', linewidth=2)
axes[1, 0].set_xlabel('Number of Algorithms', fontsize=12)
axes[1, 0].set_title('Algorithms by Category', fontsize=14, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3, axis='x')

# Plot 4: Key characteristics
characteristics = [
    'Uses Gradient',
    'Uses Hessian',
    'Handles Constraints',
    'Stochastic',
    'Adaptive Step Size'
]
char_counts = [14, 2, 4, 2, 3]  # Manual count

axes[1, 1].barh(characteristics, char_counts, color=sns.color_palette('viridis', len(characteristics)),
                edgecolor='black', linewidth=1.5)
axes[1, 1].set_xlabel('Number of Algorithms', fontsize=12)
axes[1, 1].set_title('Key Characteristics', fontsize=14, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print('\n' + '='*80)
print('All 16 optimization algorithms implemented successfully!')
print('Each includes:')
print('  ✓ Mathematical formulation')
print('  ✓ Clean Python implementation')
print('  ✓ Comprehensive visualizations')
print('  ✓ Real example datasets')
print('  ✓ Detailed explanations')
print('='*80)